Docker Compose 起動エラー調査用ノートブック

このノートブックは各サービスごとに1セルを割り当て、各エンドポイントへ順にリクエストを送りステータスとレスポンスを確認します。

In [1]:
# --- ライブラリのインポート ---
import requests
import json
import time
import traceback
import concurrent.futures
import pandas as pd
from IPython.display import display

# 設定
REQUEST_TIMEOUT = 5  # seconds
RETRY_COUNT = 1
RETRY_DELAY = 1

print("Imports done. REQUEST_TIMEOUT=", REQUEST_TIMEOUT)

Imports done. REQUEST_TIMEOUT= 5


In [2]:
# --- SERVICES 定義 ---
SERVICES = {
    "1-coordinator-ring": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "2-quorum-consistency": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "3-sharding-replica": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "4-distributed-lock": {
        "port": 8000,
        "endpoints": {
            "health": "/stats",
            "acquire": "/acquire",
            "release": "/release"
        },
        "test_data": {"resource": "test_resource", "owner": "test_client"}
    },
    "5-cache-aside": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "get": "/get",
            "set": "/set"
        },
        "test_data": {"entity_type": "user", "entity_id": "1", "data": {"name": "test"}}
    },
    "6-bloom-sstable": {
        "port": 5000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "7-rate-limiting": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "check": "/check_rate"
        },
        "test_data": {"user_id": "test_user"}
    },
    "8-line-streams": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "produce": "/produce",
            "consume": "/consume"
        },
        "test_data": {"message": "test_message", "consumer": "test_consumer"}
    },
    "9-session-store": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "login": "/login",
            "me": "/me"
        },
        "test_data": {"username": "test_user"}
    },
    "10-leaderboard": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "score": "/score",
            "top": "/top/10"
        },
        "test_data": {"user_id": "test_user", "score": 100}
    }
}

print("SERVICES configured with", len(SERVICES), "services")

SERVICES configured with 10 services


In [3]:
# --- ヘルパー関数: send_request ---
from requests.exceptions import RequestException

def try_parse_json(resp):
    try:
        return resp.json()
    except Exception:
        return None


def send_request(service_name, path, method='GET', json_body=None, params=None, timeout=REQUEST_TIMEOUT, session=None):
    cfg = SERVICES.get(service_name)
    port = cfg.get('port') if cfg else None
    url = f"http://localhost:{port}{path}"
    method = method.upper()
    last_exc = None

    for attempt in range(1, RETRY_COUNT+1):
        try:
            start = time.time()
            if session is not None:
                resp = session.request(method, url, json=json_body, params=params, timeout=timeout)
            else:
                resp = requests.request(method, url, json=json_body, params=params, timeout=timeout)
            elapsed = time.time() - start

            parsed = try_parse_json(resp)
            body = parsed if parsed is not None else resp.text

            return {
                'service': service_name,
                'url': url,
                'method': method,
                'status_code': resp.status_code,
                'body': body,
                'elapsed': elapsed,
                'error': None
            }
        except RequestException as e:
            last_exc = e
            time.sleep(RETRY_DELAY)
        except Exception as e:
            last_exc = e
            time.sleep(RETRY_DELAY)

    # all retries failed
    return {
        'service': service_name,
        'url': url,
        'method': method,
        'status_code': None,
        'body': None,
        'elapsed': None,
        'error': ''.join(traceback.format_exception_only(type(last_exc), last_exc)).strip()
    }

print('Helper functions defined')

Helper functions defined


In [4]:
# --- 1-coordinator-ring のテスト ---
service = '1-coordinator-ring'
cfg = SERVICES[service]
print('Testing', service)

# health
r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

# write
r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
print(r_write)

# read (use params)
params = {'key': cfg['test_data'].get('key')}
r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
print(r_read)

Testing 1-coordinator-ring
{'service': '1-coordinator-ring', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.016971826553344727, 'error': None}
{'service': '1-coordinator-ring', 'url': 'http://localhost:8000/write', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.0034456253051757812, 'error': None}
{'service': '1-coordinator-ring', 'url': 'http://localhost:8000/read', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n

In [5]:
# --- 2-quorum-consistency のテスト ---
service = '2-quorum-consistency'
cfg = SERVICES[service]
print('Testing', service)

r = send_request(service, cfg['endpoints']['health'], method='GET')
print(r)

r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
print(r_write)

time.sleep(0.5)
params = {'key': cfg['test_data'].get('key')}
r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
print(r_read)

Testing 2-quorum-consistency
{'service': '2-quorum-consistency', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.0049381256103515625, 'error': None}
{'service': '2-quorum-consistency', 'url': 'http://localhost:8000/write', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.0020852088928222656, 'error': None}
{'service': '2-quorum-consistency', 'url': 'http://localhost:8000/read', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try aga

In [6]:
# --- 3-sharding-replica のテスト ---
service = '3-sharding-replica'
cfg = SERVICES[service]
print('Testing', service)

try:
    r = send_request(service, cfg['endpoints']['health'], method='GET')
    print(r)

    r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
    print(r_write)

    params = {'key': cfg['test_data'].get('key')}
    r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
    print(r_read)
except Exception as e:
    print('Exception during test:', e)
    traceback.print_exc()

Testing 3-sharding-replica
{'service': '3-sharding-replica', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.004817008972167969, 'error': None}
{'service': '3-sharding-replica', 'url': 'http://localhost:8000/write', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.004187107086181641, 'error': None}
{'service': '3-sharding-replica', 'url': 'http://localhost:8000/read', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'

In [7]:
# --- 4-distributed-lock のテスト ---
service = '4-distributed-lock'
cfg = SERVICES[service]
print('Testing', service)

r = send_request(service, cfg['endpoints']['health'], method='GET')
print('health:', r)

r_acq = send_request(service, cfg['endpoints']['acquire'], method='POST', json_body=cfg['test_data'])
print('acquire:', r_acq)

r_rel = send_request(service, cfg['endpoints']['release'], method='POST', json_body=cfg['test_data'])
print('release:', r_rel)

Testing 4-distributed-lock
health: {'service': '4-distributed-lock', 'url': 'http://localhost:8000/stats', 'method': 'GET', 'status_code': 200, 'body': {'bloom_items': 0, 'l1_cache': 0, 'l2_cache': 0, 'memtable_size': 0, 'sstable_files': 0}, 'elapsed': 0.006173849105834961, 'error': None}
acquire: {'service': '4-distributed-lock', 'url': 'http://localhost:8000/acquire', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.002305746078491211, 'error': None}
release: {'service': '4-distributed-lock', 'url': 'http://localhost:8000/release', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spel

In [8]:
# --- 5-cache-aside のテスト ---
service = '5-cache-aside'
cfg = SERVICES[service]
print('Testing', service)

r = send_request(service, cfg['endpoints']['health'], method='GET')
print('health:', r)

# set
r_set = send_request(service, cfg['endpoints']['set'], method='POST', json_body=cfg['test_data'])
print('set:', r_set)

# get
params = {'entity_type': cfg['test_data']['entity_type'], 'entity_id': cfg['test_data']['entity_id']}
r_get = send_request(service, cfg['endpoints']['get'], method='GET', params=params)
print('get:', r_get)

# basic assertion
if r_get.get('body') is not None:
    if isinstance(r_get['body'], dict) and r_get['body'].get('data') == cfg['test_data']['data']:
        print('GET returned expected data')
    else:
        print('GET did not return expected data:', r_get['body'])

Testing 5-cache-aside
health: {'service': '5-cache-aside', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.0033080577850341797, 'error': None}
set: {'service': '5-cache-aside', 'url': 'http://localhost:8000/set', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.003488779067993164, 'error': None}
get: {'service': '5-cache-aside', 'url': 'http://localhost:8000/get', 'method': 'GET', 'status_code': 200, 'body': {'found': False}, 'elapsed': 0.0033109188079833984, 'error': None}
GET did not return expected data: {'found': False}


In [9]:
# --- 6-bloom-sstable のテスト ---
service = '6-bloom-sstable'
cfg = SERVICES[service]
print('Testing', service)

r = send_request(service, cfg['endpoints']['health'], method='GET')
print('health:', r)

r_write = send_request(service, cfg['endpoints']['write'], method='POST', json_body=cfg['test_data'])
print('write:', r_write)

params = {'key': cfg['test_data'].get('key')}
r_read = send_request(service, cfg['endpoints']['read'], method='GET', params=params)
print('read:', r_read)

Testing 6-bloom-sstable
health: {'service': '6-bloom-sstable', 'url': 'http://localhost:5000/health', 'method': 'GET', 'status_code': 403, 'body': '', 'elapsed': 0.007149934768676758, 'error': None}
write: {'service': '6-bloom-sstable', 'url': 'http://localhost:5000/write', 'method': 'POST', 'status_code': 403, 'body': '', 'elapsed': 0.0020270347595214844, 'error': None}
read: {'service': '6-bloom-sstable', 'url': 'http://localhost:5000/read', 'method': 'GET', 'status_code': 403, 'body': '', 'elapsed': 0.002405881881713867, 'error': None}


In [10]:
# --- 7-rate-limiting のテスト ---
service = '7-rate-limiting'
cfg = SERVICES[service]
print('Testing', service)

r = send_request(service, cfg['endpoints']['health'], method='GET')
print('health:', r)

# call check_rate multiple times
results = []
for i in range(5):
    r_check = send_request(service, cfg['endpoints']['check'], method='GET', params=cfg['test_data'])
    print(f'check attempt {i+1}:', r_check)
    results.append(r_check)
    time.sleep(0.2)

Testing 7-rate-limiting
health: {'service': '7-rate-limiting', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.0055277347564697266, 'error': None}
check attempt 1: {'service': '7-rate-limiting', 'url': 'http://localhost:8000/check_rate', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.005059957504272461, 'error': None}
check attempt 2: {'service': '7-rate-limiting', 'url': 'http://localhost:8000/check_rate', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please 

In [11]:
# --- 8-line-streams のテスト ---
service = '8-line-streams'
cfg = SERVICES[service]
print('Testing', service)

r = send_request(service, cfg['endpoints']['health'], method='GET')
print('health:', r)

# produce
r_prod = send_request(service, cfg['endpoints']['produce'], method='POST', json_body=cfg['test_data'])
print('produce:', r_prod)

# consume
r_consume = send_request(service, cfg['endpoints']['consume'], method='GET', params={'consumer': cfg['test_data'].get('consumer')})
print('consume:', r_consume)

Testing 8-line-streams
health: {'service': '8-line-streams', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.0058710575103759766, 'error': None}
produce: {'service': '8-line-streams', 'url': 'http://localhost:8000/produce', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.004259824752807617, 'error': None}
consume: {'service': '8-line-streams', 'url': 'http://localhost:8000/consume', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and t

In [12]:
# --- 9-session-store のテスト ---
service = '9-session-store'
cfg = SERVICES[service]
print('Testing', service)

session = requests.Session()

r = send_request(service, cfg['endpoints']['health'], method='GET', session=session)
print('health:', r)

# login
r_login = send_request(service, cfg['endpoints']['login'], method='POST', json_body=cfg['test_data'], session=session)
print('login:', r_login)

# me
r_me = send_request(service, cfg['endpoints']['me'], method='GET', session=session)
print('me:', r_me)

Testing 9-session-store
health: {'service': '9-session-store', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.003179788589477539, 'error': None}
login: {'service': '9-session-store', 'url': 'http://localhost:8000/login', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.003673076629638672, 'error': None}
me: {'service': '9-session-store', 'url': 'http://localhost:8000/me', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</

In [13]:
# --- 10-leaderboard のテスト ---
service = '10-leaderboard'
cfg = SERVICES[service]
print('Testing', service)

r = send_request(service, cfg['endpoints']['health'], method='GET')
print('health:', r)

# score
r_score = send_request(service, cfg['endpoints']['score'], method='POST', json_body=cfg['test_data'])
print('score:', r_score)

# top
r_top = send_request(service, cfg['endpoints']['top'], method='GET')
print('top:', r_top)

# check if user in top
if isinstance(r_top.get('body'), list):
    found = any((item.get('user_id') == cfg['test_data']['user_id']) for item in r_top['body'])
    print('User present in top:', found)

Testing 10-leaderboard
health: {'service': '10-leaderboard', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.003876924514770508, 'error': None}
score: {'service': '10-leaderboard', 'url': 'http://localhost:8000/score', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.0033049583435058594, 'error': None}
top: {'service': '10-leaderboard', 'url': 'http://localhost:8000/top/10', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

In [14]:
# --- 全サービス一括実行とサマリ ---
results = []

for service_name in SERVICES.keys():
    cfg = SERVICES[service_name]
    print('\n=== SERVICE:', service_name, '===')

    # use a session for services that need it
    session = requests.Session()

    for ep_name, path in cfg['endpoints'].items():
        method = 'GET'
        if ep_name in ('write', 'set', 'acquire', 'release', 'produce', 'login', 'score'):
            method = 'POST'

        json_body = None
        params = None
        if method == 'POST':
            json_body = cfg.get('test_data')
        else:
            # construct params from common key names
            td = cfg.get('test_data', {})
            if 'key' in td:
                params = {'key': td['key']}
            elif 'entity_type' in td and 'entity_id' in td:
                params = {'entity_type': td['entity_type'], 'entity_id': td['entity_id']}
            elif 'consumer' in td:
                params = {'consumer': td['consumer']}
            elif 'user_id' in td:
                params = {'user_id': td['user_id']}

        r = send_request(service_name, path, method=method, json_body=json_body, params=params, session=session)
        results.append(r)
        print(r)

# summary
df = pd.DataFrame(results)
if 'status_code' in df.columns:
    df['success'] = df['status_code'].apply(lambda s: 200 <= s < 300 if pd.notnull(s) else False)

display(df)

failed = df[~df['success']]
if not failed.empty:
    print('\nFailures:')
    display(failed)
else:
    print('\nAll endpoints returned 2xx statuses where applicable')


=== SERVICE: 1-coordinator-ring ===
{'service': '1-coordinator-ring', 'url': 'http://localhost:8000/health', 'method': 'GET', 'status_code': 503, 'body': {'message': 'Error 111 connecting to localhost:6379. Connection refused.', 'status': 'error'}, 'elapsed': 0.003196239471435547, 'error': None}
{'service': '1-coordinator-ring', 'url': 'http://localhost:8000/write', 'method': 'POST', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n', 'elapsed': 0.0030739307403564453, 'error': None}
{'service': '1-coordinator-ring', 'url': 'http://localhost:8000/read', 'method': 'GET', 'status_code': 404, 'body': '<!doctype html>\n<html lang=en>\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try ag

,service,url,method,status_code,body,elapsed,error,success
0,1-coordinator-ring,http://localhost:8000/health,GET,503,{'message': 'Error 111 connecting to localhost...,0.003196,None,False
1,1-coordinator-ring,http://localhost:8000/write,POST,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003074,None,False
2,1-coordinator-ring,http://localhost:8000/read,GET,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003205,None,False
3,2-quorum-consistency,http://localhost:8000/health,GET,503,{'message': 'Error 111 connecting to localhost...,0.003354,None,False
4,2-quorum-consistency,http://localhost:8000/write,POST,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003008,None,False
5,2-quorum-consistency,http://localhost:8000/read,GET,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.002656,None,False
6,3-sharding-replica,http://localhost:8000/health,GET,503,{'message': 'Error 111 connecting to localhost...,0.002663,None,False
7,3-sharding-replica,http://localhost:8000/write,POST,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003096,None,False
8,3-sharding-replica,http://localhost:8000/read,GET,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.002753,None,False
9,4-distributed-lock,http://localhost:8000/stats,GET,200,"{'bloom_items': 0, 'l1_cache': 0, 'l2_cache': ...",0.003513,None,True



Failures:


,service,url,method,status_code,body,elapsed,error,success
0,1-coordinator-ring,http://localhost:8000/health,GET,503,{'message': 'Error 111 connecting to localhost...,0.003196,None,False
1,1-coordinator-ring,http://localhost:8000/write,POST,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003074,None,False
2,1-coordinator-ring,http://localhost:8000/read,GET,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003205,None,False
3,2-quorum-consistency,http://localhost:8000/health,GET,503,{'message': 'Error 111 connecting to localhost...,0.003354,None,False
4,2-quorum-consistency,http://localhost:8000/write,POST,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003008,None,False
5,2-quorum-consistency,http://localhost:8000/read,GET,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.002656,None,False
6,3-sharding-replica,http://localhost:8000/health,GET,503,{'message': 'Error 111 connecting to localhost...,0.002663,None,False
7,3-sharding-replica,http://localhost:8000/write,POST,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003096,None,False
8,3-sharding-replica,http://localhost:8000/read,GET,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.002753,None,False
10,4-distributed-lock,http://localhost:8000/acquire,POST,404,<!doctype html>\n<html lang=en>\n<title>404 No...,0.003162,None,False


実行方法:

- 必要なパッケージをインストール: pip install requests pandas
- Jupyter で開いて各セルを順に実行するか、一括で実行して結果を確認してください。
- docker-compose のサービスが別ポートにバインドされている場合は `SERVICES` の port を調整してください。

備考: タイムアウトや接続拒否の詳細はセル出力に表示されます。ネットワークやコンテナのログと合わせて調査してください。